In [2]:
import os
import sys
sys.path.append(os.path.abspath('..'))

import torch
from torch.utils.data import DataLoader
from transformer_lens import HookedTransformer
from transformer_lens import utils as tutils
from transformer_lens.evals import make_pile_data_loader, evaluate_on_dataset

from functools import partial
from datasets import load_dataset
from tqdm import tqdm

import einops
import json

from typing import List, Callable, Union, Optional, Literal
from collections import defaultdict

from sae_lens import SAE
# from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes
# from sae_lens import SparseAutoencoder, ActivationsStore

# from steering.eval_utils import evaluate_completions
from steering.utils import normalise_decoder, text_to_sae_feats
from steering.patch import generate, scores_2d, patch_resid

# from sae_vis.data_config_classes import SaeVisConfig
# from sae_vis.data_storing_fns import SaeVisData

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
from scipy import stats

torch.set_grad_enabled(False)

In [3]:
g2_forward_indices = torch.load("../effects/g2_top_is.pt", map_location=torch.device("cpu"))
g2_forward_values = torch.load("../effects/g2_top_vs.pt", map_location=torch.device("cpu"))

g_forward_indices = torch.load("top_is.pt", map_location=torch.device("cpu"))
g_forward_values = torch.load("top_vs.pt", map_location=torch.device("cpu"))

/tmp/ipykernel_29008/420386180.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  g2_forward_indices = torch.load("../effects/g2_top_is.pt", map_location=torch.device("cpu"

In [4]:
num_features = g_forward_indices.shape[0]
# Create a square adjacency matrix filled with zeros
g_adjacency_matrix = torch.zeros((num_features, num_features))

for i in range(num_features):
    # Get non-zero indices and values
    non_zero_mask = g_forward_values[i] != 0
    indices = g_forward_indices[i][non_zero_mask]
    values = g_forward_values[i][non_zero_mask]
    
    # Set the values in the adjacency matrix
    g_adjacency_matrix[i, indices] = values

g2_adjacency_matrix = torch.zeros((num_features, num_features))

for i in range(num_features):
    non_zero_mask = g2_forward_values[i] != 0
    indices = g2_forward_indices[i][non_zero_mask]
    values = g2_forward_values[i][non_zero_mask]
    
    g2_adjacency_matrix[i, indices] = values

In [12]:
g2_london = 14455
g_london = 10138

g2_anger = 4111
g_anger = 1062

g2_wedding_1 = 4230
g2_wedding_2 = 6021
g_wedding_1 = 8406
g_wedding_2 = 2378

In [7]:
print(g2_adjacency_matrix[g2_london, :].topk(10))
print(g_adjacency_matrix[g_london, :].topk(10))

torch.return_types.topk(
values=tensor([0.5479, 0.2494, 0.2442, 0.1779, 0.1537, 0.1027, 0.0998, 0.0996, 0.0986,
        0.0951]),
indices=tensor([6810, 4354, 7507, 8675, 3928, 8464, 8223, 8645,  847, 3760]))
torch.return_types.topk(
values=tensor([1.0038, 0.5537, 0.2807, 0.2729, 0.1502, 0.1440, 0.1255, 0.1239, 0.1153,
        0.1104]),
indices=tensor([10138, 10655,  4343, 12090,  5523, 11444,  9104, 11912, 13568,  7329]))


In [10]:
print(g2_adjacency_matrix[g2_anger, :].topk(10))
print(g_adjacency_matrix[g_anger, :].topk(10))

torch.return_types.topk(
values=tensor([0.5233, 0.4914, 0.3599, 0.3028, 0.2941, 0.2929, 0.2885, 0.2346, 0.1975,
        0.1929]),
indices=tensor([ 7507,  2620,  8806,  4111,  2009,  5511,   718, 12037,  5605,  4790]))
torch.return_types.topk(
values=tensor([1.0049, 0.4441, 0.3163, 0.2815, 0.2554, 0.2284, 0.2196, 0.2130, 0.2093,
        0.1897]),
indices=tensor([ 1062, 10871,  1725, 15231,  2813, 13524,   875,  2482, 15989,  2458]))


In [ ]:
# wedding
print(g2_adjacency_matrix[g2_wedding_1, :].topk(10))
print(g_adjacency_matrix[g_wedding_1, :].topk(10))
print(g2_adjacency_matrix[g2_wedding_2, :].topk(10))
print(g_adjacency_matrix[g_wedding_2, :].topk(10))